**Objetivo.** Armar una tabla que tenga mesa, votos a cada partido para intendente, votos a cada partido para concejal, votos a cada alianza para intendente, votos a cada alianza para concejal, votos en blanco, nulo, etc, votos totales, y estimación del voto cruzado.

Los datos de la elección los bajé de https://elecciones2017.corrientes.gov.ar/ con un scrapper hecho en PHP artesanalmente. El código está [acá](https://drive.google.com/open?id=0B_6HwOsRj0TEblVFeGw0UkxoNjA). Los bajé en formato JSON. Son tres archivos, con resultados por lista, por alianza para intendente, y por alianza para concejales. También uso la [tabla de mesas](https://elecciones2017.corrientes.gov.ar/tabla_mesas.php) convertida a csv.

Primero cargo los datos JSON:

In [1]:
import json

with open('resultados.json') as data_file:    
    resultados = json.load(data_file)
with open('alianzas_intendentes.json') as data_file:    
    alianzas_intendentes = json.load(data_file)
with open('alianzas_concejales.json') as data_file:    
    alianzas_concejales = json.load(data_file)

Armo un dataframe en Pandas cuyas filas son las mesas de votación y las columnas son los resultados por lista (tanto para intendente como para concejales). Le sumo los resultados para las alianzas para intendente y concejales. Luego la información de las mesas (su circuito y si tiene BUE o no). Finalmente, lo guardo a un csv (`todo.csv`) y muestro las primeras filas.

In [2]:
import pandas as pd
import html

columnas = ['mesa']
cols = {'mesa': []}
res = {}

for mesa, fila in resultados.items():
    cols['mesa'].append(int(mesa))
    for d in fila:
        partido = html.unescape(d['Partidos y/o Alianzas'])
        intentente = d['Intendente/Vice']
        concejales = d['Concejales']
        part_int = partido + ' 1.Intendente/Vice'
        part_conc = partido + ' 2.Concejales'
        if part_int not in cols:
            cols[part_int] = []
            columnas.append(part_int)
        if part_conc not in cols:
            cols[part_conc] = []
            columnas.append(part_conc)
        cols[part_int].append(intentente)
        cols[part_conc].append(concejales)

df = pd.DataFrame(cols)
df = df[columnas]
df = df.sort_values(by='mesa')
res_df = df.copy()

In [3]:
columnas = ['mesa']
cols = {'mesa': []}
res = {}

for mesa, fila in alianzas_intendentes.items():
    cols['mesa'].append(int(mesa))
    for d in fila:
        partido = html.unescape(d['Alianzas Intendente/Vice'])
        intentente = d['Total Votos']
        part_int = partido + ' 1.Alianzas Intendente/Vice'
        if part_int not in cols:
            cols[part_int] = []
            columnas.append(part_int)
        cols[part_int].append(intentente)

df = pd.DataFrame(cols)
df = df[columnas]
df = df.sort_values(by='mesa')
al_int_df = df.copy()

In [4]:
columnas = ['mesa']
cols = {'mesa': []}
res = {}

for mesa, fila in alianzas_concejales.items():
    cols['mesa'].append(int(mesa))
    for d in fila:
        partido = html.unescape(d['Alianzas Concejales'])
        concejales = d['Total Votos']
        part_conc = partido + ' 2.Alianzas Concejales'
        if part_conc not in cols:
            cols[part_conc] = []
            columnas.append(part_conc)
        cols[part_conc].append(concejales)

df = pd.DataFrame(cols)
df = df[columnas]
df = df.sort_values(by='mesa')
al_con_df = df.copy()

In [5]:
mesas_df = pd.read_csv('mesas.csv')
mesas_df.rename(columns={'MESA':'mesa'}, inplace=True)
mesas_df.drop('ESTADO', axis=1, inplace=True)
mesas_df.head()
mesa, bue = [], []
for row in mesas_df.values:
    xs = row[0].split()
    mesa.append(int(xs[0]))
    bue.append(len(xs)-1)
mesas_df['mesa'] = pd.Series(mesa, index=mesas_df.index)
mesas_df['bue'] = pd.Series(bue, index=mesas_df.index)

In [6]:
result = pd.merge(mesas_df, res_df, on='mesa')
result = pd.merge(result, al_int_df, on='mesa')
result = pd.merge(result, al_con_df, on='mesa')
result.to_csv('todo.csv')
result.head()

,mesa,Departamento,Municipio,Circuito,bue,1 - Movimiento Integracion y Desarrollo (1) 1.Intendente/Vice,1 - Movimiento Integracion y Desarrollo (1) 2.Concejales,2 - Justicialista (2) 1.Intendente/Vice,2 - Justicialista (2) 2.Concejales,3 - Union Civica Radical (3) 1.Intendente/Vice,...,Total Votos Válidos 2.Concejales,Votos Generales 1.Intendente/Vice,Votos Generales 2.Concejales,Frente Para la Victoria (201) 1.Alianzas Intendente/Vice,Encuentro por Corrientes-ECO+Cambiemos (203) 1.Alianzas Intendente/Vice,Haciendo Corrientes (205) 1.Alianzas Intendente/Vice,Ciudadanos Comprometidos (200) 2.Alianzas Concejales,Frente Para la Victoria (201) 2.Alianzas Concejales,Encuentro por Corrientes-ECO+Cambiemos (204) 2.Alianzas Concejales,Haciendo Corrientes (205) 2.Alianzas Concejales
0,1,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,8,6,23,20,42,...,227,242,242,9,145,78,5,9,119,67
1,2,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,11,11,26,22,37,...,214,243,235,7,138,71,7,5,120,65
2,3,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,2,2,16,14,45,...,231,274,269,6,157,76,13,7,117,65
3,4,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,11,8,32,27,26,...,212,226,217,9,139,73,8,8,120,60
4,5,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,10,8,17,14,66,...,228,246,246,6,166,62,3,9,141,53


Calculo los votos cruzados para cada mesa $i$. La primera medida es $$\frac12\sum_{j\in L}|I_{ij}-C_{ij}|,$$ donde $L$ es el conjunto de listas, $I_{ij}$ es la cantidad de votos a la lista $j$ en la mesa $i$ para intendente, y $C_{ij}$ es la cantidad de votos a la lista $j$ en la mesa $i$ para concejales.

La segunda medida es $$\frac12\sum_a\left|\sum_{j\in L_a}(I_{ij}-C_{ij})\right|,$$ donde $a$ recorre los candidatos a intendente (o las alianzas de listas para la intendencia), $L_a$ es el conjunto de listas que llevaron al candidato $a$ para intendente, y $I_{ij}$ y $C_{ij}$ son como antes. Lo que intenta medir esto es los votos cruzados entre alianzas, no entre listas. (Es más restrictiva que la medición anterior. Por desigualdad triangular va a ser siempre menor o igual que la anterior.)

In [7]:
df = result
HASTA = 79 #83 para incluir blancos y nulos
cruzado = []
for i in range(len(df)):
    s = 0
    for j in range(5,HASTA,2):
        a, b = df.iloc[i,j], df.iloc[i,j+1]
        if a == '/////': a = '0'
        if b == '/////': b = '0'
        a, b = int(a), int(b)
        s += abs(a-b)
    cruzado.append(s/2)
df['voto_cruzado'] = pd.Series(cruzado, index=df.index)

In [8]:
import re
txt = '''
<span id="Span1" style="display:none">
<b>Frente Para la Victoria (201)                                                                        </b><br>* Convocatoria Popular (21)                                                                           : 1 votos.<br>* KOLINA (192)                                                                                        : 8 votos.<br><b>Encuentro por Corrientes-ECO+Cambiemos (203)                                                         </b><br>* Union Civica Radical (3)                                                                            : 42 votos.<br>* Accion por la Republica (44)                                                                        : 3 votos.<br>* Autonomista (52)                                                                                    : 8 votos.<br>* Nueva Dirigencia Corrientes (67)                                                                    : 1 votos.<br>* Proyecto Corrientes (57)                                                                            : 2 votos.<br>* Conservador Popular (31)                                                                            : 0 votos.<br>* Democrata Progresista (9)                                                                           : 1 votos.<br>* Popular (169)                                                                                       : 19 votos.<br>* Afirmacion para una Republica Igualitaria ARI  (47)                                                 : 10 votos.<br>* Compromiso Correntino (175)                                                                         : 3 votos.<br>* Es Posible (6)                                                                                      : 4 votos.<br>* Partido Socialista (50)                                                                             : 0 votos.<br>* Accion Popular de los Trabajadores (34)                                                             : 2 votos.<br>* Union Popular (25)                                                                                  : 0 votos.<br>* Union Celeste y Blanco (173)                                                                        : 12 votos.<br>* PRO - Propuesta Republicana (64)                                                                    : 11 votos.<br>* Accion por Corrientes (14)                                                                          : 5 votos.<br>* Unidos por Corrientes (15)                                                                          : 0 votos.<br>* Federal (8)                                                                                         : 1 votos.<br>* Laborista Autonomo (17)                                                                             : 0 votos.<br>* ELI - Encuentro Liberal (18)                                                                        : 18 votos.<br>* FE (23)                                                                                             : 3 votos.<br>* Instrumento Electoral por la Unidad Popular (72)                                                    : 0 votos.<br><b>Haciendo Corrientes (205)                                                                            </b><br>* de la Victoria (54)                                                                                 : 3 votos.<br>* Union del Centro Democratico (20)                                                                   : 1 votos.<br>* Justicialista (2)                                                                                   : 23 votos.<br>* Nuevo (90)                                                                                          : 7 votos.<br>* Movimiento Integracion y Desarrollo (1)                                                             : 8 votos.<br>* Democrata Cristiano (5)                                                                             : 0 votos.<br>* Crecer con Todos (170)                                                                              : 14 votos.<br>* Movimiento Siempre Corrientes (4)                                                                   : 7 votos.<br>* de la Concertacion Forja (7)                                                                        : 0 votos.<br>* Cambio Popular (19)                                                                                 : 1 votos.<br>* Renovador Federal (10)                                                                              : 5 votos.<br>* Ladrilleros Unidos (CAPITAL) (11)                                                                   : 9 votos.<br></span>
'''
ls = []
for n in re.findall(r'\((\d+)\)', txt):
    n = int(n)
    if n >= 200:
        ls.append(set())
    else:
        ls[-1].add(n)
#print(ls)
ls_ = {i:{} for i in range(len(ls))}
for i, col in enumerate(df.columns):
    a = re.search(r'\((\d+)\)', col)
    if not a: continue
    n = int(a.group(1))
    if n >= 200: break
    for j in range(len(ls)):
        if n in ls[j]:
            if n not in ls_[j]:
                #print(j, '->', i, '->', col)
                ls_[j][n] = i
#print(ls_)
get = lambda i, j: int(df.iloc[i,j]) if df.iloc[i,j] != '/////' else 0
cruzado = []
for i in range(len(df)):
    s = [0 for _ in range(len(ls))]
    for j in range(len(ls)):
        s[j] = abs(sum(((get(i,k) - get(i,k+1)) for u, k in ls_[j].items())))
    cruzado.append(sum(s)/2)
df['voto_cruzado_revisado'] = pd.Series(cruzado, index=df.index)
df.to_csv('todo.csv')
df.head()

,mesa,Departamento,Municipio,Circuito,bue,1 - Movimiento Integracion y Desarrollo (1) 1.Intendente/Vice,1 - Movimiento Integracion y Desarrollo (1) 2.Concejales,2 - Justicialista (2) 1.Intendente/Vice,2 - Justicialista (2) 2.Concejales,3 - Union Civica Radical (3) 1.Intendente/Vice,...,Votos Generales 2.Concejales,Frente Para la Victoria (201) 1.Alianzas Intendente/Vice,Encuentro por Corrientes-ECO+Cambiemos (203) 1.Alianzas Intendente/Vice,Haciendo Corrientes (205) 1.Alianzas Intendente/Vice,Ciudadanos Comprometidos (200) 2.Alianzas Concejales,Frente Para la Victoria (201) 2.Alianzas Concejales,Encuentro por Corrientes-ECO+Cambiemos (204) 2.Alianzas Concejales,Haciendo Corrientes (205) 2.Alianzas Concejales,voto_cruzado,voto_cruzado_revisado
0,1,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,8,6,23,20,42,...,242,9,145,78,5,9,119,67,20.5,8.5
1,2,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,11,11,26,22,37,...,235,7,138,71,7,5,120,65,13.0,7.0
2,3,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,2,2,16,14,45,...,269,6,157,76,13,7,117,65,14.0,7.0
3,4,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,11,8,32,27,26,...,217,9,139,73,8,8,120,60,15.5,9.5
4,5,241 -- CAPITAL,1 -- CAPITAL,1 -- CAPITAL,0,10,8,17,14,66,...,246,6,166,62,3,9,141,53,15.0,6.0
